In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statistics import mean
import matplotlib.font_manager

from RoomMeetingAssignments import *
from PeerConnection import *

from rmsops import *

fontName = 'YS Text'
fontColor = '#FF6427'
fontColor2 = '#000000'

font_files = matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype')
for font_file in font_files:
    try: 
        matplotlib.font_manager.fontManager.addfont(font_file)
    except Exception:
        print('failed to add font ' + font_file)
        #ignore
font = {'family' : 'YS Text',
        'size'   : 16}

matplotlib.rc('font', **font)

failed to add font /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf


In [2]:
#to speed up testing
MAX_NUM_ROWS=50000
# MAX_NUM_ROWS=10000000000

MEDIASERVER_IDLE_TIMEOUT = 60

#model parameters
ISLAND_SIZES = [50] * 5
RECONNECT_DOWNTIME_SECONDS = 10

#parameters to optimize
GRACE_PERIOD_SECONDS = 1800

#disruptions are done one island at a time. When one rolls out, other islands stay 100% available
DISRUPTION_BUDGET = 50

In [3]:
calls  = pd.read_csv('calls_data_week.tsv', header=0, names=['msid','peer_id', 'room_id', 'rsid', 'ts_connected', 'ts_joined', 'ts_leave', 'ts_offer'], delimiter='\t', nrows=MAX_NUM_ROWS)
print(f"Room Media Sessions loaded: {calls.shape[0]}")
# calls.head(15)

Room Media Sessions loaded: 50000


In [4]:
import rms_tests
rms_tests.run_tests(MEDIASERVER_IDLE_TIMEOUT)

In [5]:
rmsConnections = [None] * calls.shape[0]
peerConnections: dict[str, PeerConnection] = dict()
#to be split by concurrent groups later. needed to assign media servers to these sessions
sameRoomRMSConnections: dict[str, list[RMSConnection]] = dict()
errorsConnected = []
errorsJoined = []
errorsLeave = []

def registerRMSConnection(rmsc: RMSConnection):
    pcKey = f"{rmsc.room_id}_{rmsc.peer_id}"
    pc: PeerConnection = None
    if(pcKey not in peerConnections):
        pc = PeerConnection([rmsc])
        peerConnections[pcKey] = pc
    else:
        pc = peerConnections[pcKey]
    pc.ts_joined = min(pc.ts_joined, rmsc.ts_joined)
    pc.ts_connected = min(pc.ts_connected, rmsc.ts_connected)
    pc.ts_leave = max(pc.ts_leave, rmsc.ts_leave)
    pc.rmsConnections.append(rmsc)

    sameRoom = None
    if(rmsc.room_id not in sameRoomRMSConnections):
        sameRoom = []
        sameRoomRMSConnections[rmsc.room_id] = sameRoom
    else:
        sameRoom = sameRoomRMSConnections[rmsc.room_id]
    sameRoom.append(rmsc)

In [6]:
for index in range(0, calls.shape[0]):
    try:
        rmsc = RMSConnection(
            calls['room_id'][index],
            calls['peer_id'][index],
            calls['msid'][index],
            calls['rsid'][index],
            calls['ts_joined'][index],
            calls['ts_connected'][index],
            calls['ts_leave'][index]
        )
    except RMSConnectionCreationException as e:
        match e.reason:
            case "joined":
                errorsJoined.append(index)
            case "connected":
                errorsConnected.append(index)
            case "leave":
                errorsLeave.append(index)
            case _:
                raise f"unknown reason {e.reason}"
        
    # rmsc.room_id = calls['room_id'][index]
    # rmsc.peer_id = calls['peer_id'][index]
    # rmsc.msid = calls['msid'][index]
    # rmsc.rsid = calls['rsid'][index]
    
    # if(type(calls['ts_connected'][index]) == float):
    #     errorsConnected.append(index)
    #     continue
    # else:
    #     rmsc.ts_connected = datetime.strptime(calls['ts_connected'][index], '%Y-%m-%d %H:%M:%S,%f')

    # if(type(calls['ts_joined'][index]) == float):
    #     errorsJoined.append(index)
    #     continue
    # else:
    #     rmsc.ts_joined = datetime.strptime(calls['ts_joined'][index], '%Y-%m-%d %H:%M:%S,%f')

    # if(type(calls['ts_leave'][index]) == float):
    #     errorsLeave.append(index)
    #     continue
    # else:
    #     rmsc.ts_leave = datetime.strptime(calls['ts_leave'][index], '%Y-%m-%d %H:%M:%S,%f')
    
    rmsConnections[index] = rmsc
    registerRMSConnection(rmsc)

roomMeetings = splitRoomMeetings(sameRoomRMSConnections, MEDIASERVER_IDLE_TIMEOUT)

print(f"connected errors: {len(errorsConnected)}\nJoined errors: {len(errorsJoined)}\nLeave errors: {len(errorsLeave)}\nPeer connections: {len(peerConnections)}\nRoom IDs: {len(sameRoomRMSConnections)}\nRoom Meetings: {len(roomMeetings)}")
# print({k: str(peerConnections[k])  for k in list(peerConnections)[:10]})



connected errors: 3161
Joined errors: 0
Leave errors: 74
Peer connections: 45695
Room IDs: 37876
Room Meetings: 42331


In [7]:
def countRoomMeetingsAt(ts: datetime):
    result = 0
    for rm in roomMeetings:
        if(rm.ts_start <= ts and rm.ts_finish >= ts): result+=1
    return result


def countPeerConnectionsAt(ts: datetime):
    result = 0
    for rm in roomMeetings:
        if(rm.ts_start > ts or rm.ts_finish < ts): continue
        for pc in rm.peerConnections:
            if (pc.ts_joined <= ts and pc.ts_leave >= ts): result += 1
    return result
            
#countRoomMeetingsAt(datetime.strptime('2023-10-04 10:35:36,448', '%Y-%m-%d %H:%M:%S,%f'))
#countPeerConnectionsAt(datetime.strptime('2023-10-04 10:35:36,448', '%Y-%m-%d %H:%M:%S,%f'))


In [8]:

start_date=datetime.strptime('2023-10-04 10:30:00,000', '%Y-%m-%d %H:%M:%S,%f')
end_date=datetime.strptime('2023-10-04 11:30:00,000', '%Y-%m-%d %H:%M:%S,%f')

date_range = list(pd.date_range(start_date, end_date, freq='30S'))
roomMeetingsValues = list(map(lambda date: countRoomMeetingsAt(date), date_range))
concurrentPeerConnectionsValues = list(map(lambda date: countPeerConnectionsAt(date), date_range))

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(111)
ax.set_title('Room Meetings', fontsize=36, fontname=fontName)
ax.set_xlabel('DateTime', fontsize=22, fontname=fontName)
ax.set_ylabel('Room Meetings', fontsize=22, fontname=fontName)
plt.scatter(x=date_range, y=roomMeetingsValues, color=fontColor, label='Room Meetings')
ax.grid()
plt.legend(fontsize=22, loc ="upper right")

fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(111)
ax.set_title('Room Meetings', fontsize=36, fontname=fontName)
ax.set_xlabel('DateTime', fontsize=22, fontname=fontName)
ax.set_ylabel('Peer Connections', fontsize=22, fontname=fontName)
plt.scatter(x=date_range, y=concurrentPeerConnectionsValues, color=fontColor, label='Peer Connections')
ax.grid()
plt.legend(fontsize=22, loc ="upper right")